# Predicting cloud cover using neural networks

In [1]:
import numpy as np
import scipy.sparse as sp
np.random.seed(12)

import warnings
#Comment this to turn on warnings
warnings.filterwarnings('ignore')

from model_comparison_2 import model_comparison
from resample import resample
import algorithms
import matplotlib.pyplot as plt
import seaborn as sns
import netCDF4 as n
from sklearn.linear_model import LinearRegression
#from mpl_toolkits.axes_grid1 import make_axes_locatable

from utils import train_test_split
%matplotlib inline
#%matplotlib notebook

from deepNN import NeuralNetRegressor

from sklearn.neural_network import MLPRegressor
from utils import mean_squared_error, A_R2, NRMSE, transforming_predictorspace, standardicing_responce

from sklearn.preprocessing import StandardScaler

import itertools


In [2]:
# reading test
path = "./files/"
filenames = ["specific_humidity_Europa_sp.nc", "relative_humidity_Europa_sp.nc", "pressure_Europa_sp.nc",  
             "temperature_Europa_sp.nc", "total_cloud_cover_Europa_sp.nc"]


cloud = n.Dataset(path + filenames[-1], "r")
relative = n.Dataset(path + filenames[1], "r")
specific = n.Dataset(path + filenames[0], "r")
pressure = n.Dataset(path + filenames[2], "r")
temperature = n.Dataset(path + filenames[3], "r")

In [3]:
#print(cloud.variables)
tcc = cloud.variables["tcc"][:][:][:].data

# Retriving ground values, these are available at six different pressure levels. 
rel = relative.variables["r"][:][:][:][:].data
#level = relative.variables["level"][:][0].data
spe = specific.variables["q"][:][:][:][:].data

surf_pre = pressure.variables["sp"][:][:][:].data
temp = temperature.variables["t2m"][:][:][:].data

In [4]:
def logit_inv(x): # sigmoid?
    return np.exp(x)/(1+np.exp(x))

def logit(x):
    return np.log((x + 1e-12)/(1+1e-12 - x))

In [5]:
# for one certain timestep 

n_days = 7

TCC = []
REL = []
SPE = []
PRE = []
TEMP = []


for t in range(int(n_days*4)):
    TCC.append(tcc[t][:][:].flatten())
    REL.append(rel[t][0][:][:].flatten())
    SPE.append(spe[t][0][:][:].flatten())
    PRE.append(surf_pre[t][:][:].flatten())
    TEMP.append(temp[t][:][:].flatten())


In [6]:
y =(np.array(TCC).flatten())
temp = y[y<1]
y[y>1] = temp.max()
print(y.min()>0)

X = np.array([np.array(REL).flatten(), np.array(SPE).flatten(), np.array(PRE).flatten(), np.array(TEMP).flatten()])
#y = logit(np.array(TCC).flatten())

True


In [7]:
#plt.figure(figsize = (20,20))
#plt.plot(y)

In [8]:
np.array(TCC).min(), np.array(TCC).max()

(9.99866855977416e-13, 1.0000000000009999)

In [9]:
y.min(), y.max()

(9.99866855977416e-13, 0.9999694810258271)

In [10]:
np.shape(X[0]),np.shape(X[1]),np.shape(X[2]),np.shape(X[3]), np.shape(y)

((131516,), (131516,), (131516,), (131516,), (131516,))

In [11]:
y = y.reshape((len(y),1))
X = X.T

In [12]:
np.shape(X), np.shape(y)

((131516, 4), (131516, 1))

In [13]:
#X_train, X_test, y_train, y_test = train_test_split(X, y, split_size = 0.2)
import sklearn.model_selection as s
X_train, X_test, y_train, y_test = s.train_test_split(X, y, test_size = 0.2)


scaler = StandardScaler()
# Kunne brukt fit transform
fit = scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [14]:
np.shape(X_train), np.shape(X_test), np.shape(y_train),  np.shape(y_test)

((105212, 4), (26304, 4), (105212, 1), (26304, 1))

In [15]:
y_train.max(), y_train.min()

(0.9999694810258271, 9.99866855977416e-13)

# ONE HIDDEN LAYER - sigmoid - 1000 epochs

In [ ]:
nr_epochs = 1000

n_nodes = [10,30,50,100,200,300,400,500]

# Array which contains all the "calibrated models ".
results = []

for n in n_nodes:
    model = NeuralNetRegressor(n_hidden = [n],  
                               epochs=nr_epochs, 
                               eta=0.0001, # larger eta results in exploding gradients, becomes NaN.
                               shuffle = True, 
                               batch_size = 10,
                               seed = None, 
                               alpha = 0.0001, # dette er for relu ikke det samme som penaltien i scikit learn  
                               activation = 'sigmoid')

    p = model.fit(X_train, y_train, X_test, y_test)
    model.predict(X_test)
    results.append(p)
    print("Finished model: " + str(n) + " mse " + str(np.nanmean(p.eval_['valid_preform'])) + " r2 " + str(np.nanmean(p.eval_['valid_preform_r2'])))


In [ ]:
plt.figure(figsize = (20,20))

counter = 0
for model in results:
    mse_train = model.eval_['train_preform']
    mse_test = model.eval_['valid_preform']
    cost_train = model.eval_["cost_train"]
    r2_test = model.eval_['valid_preform_r2']

    ax1 = plt.subplot(211)
    plt.plot(r2_test, label = str(n_nodes[counter])) 
    plt.title("R2 score for different network architectures", fontsize = 20)
    plt.ylabel("R2 score", fontsize = 15)
    plt.legend()
    
    # Alle disse blir like.
    ax2 = plt.subplot(212)
    plt.plot(mse_test, label = str(n_nodes[counter])) 
    plt.title("MSE test for different one hidden layer architectures", fontsize = 20)
    plt.ylabel("MSE", fontsize = 15)
    plt.legend()
    
    counter += 1

plt.xlabel("Epochs", fontsize = 15)
plt.savefig("results/figures/NN_one_hidden_layer_sigmoid_1000epochs_eta0.0001_featured_scaled_7.png")

In [ ]:
print(len(results))

In [ ]:
# Look the same for all models.
error = results[0].model_error
plt.hist(error)
plt.title("Histogram error using sigmoid, 1000 epochs and one timestep of data", fontsize = 15)
plt.savefig("results/figures/NN_one_hidden_layer_sigmoid_1000epochs_eta0.0001_featured_scaled_1_error.png")
# They all look the same. 

# ONE HIDDEN LAYER - ELU - 1000 epochs

In [ ]:
nr_epochs = 1000

n_nodes = [10, 30, 50, 100, 200, 300, 400, 500]

# Array which contains all the "calibrated models ".
results = []

for n in n_nodes:
    model = NeuralNetRegressor(n_hidden = [n],  
                               epochs=nr_epochs, 
                               eta=0.0001, # larger eta results in exploding gradients, becomes NaN.
                               shuffle = True, 
                               batch_size = 10,
                               seed = None, 
                               alpha = 0.0001, # dette er for relu ikke det samme som penaltien i scikit learn  
                               activation = 'elu')

    p = model.fit(X_train, y_train, X_test, y_test)
    model.predict(X_test)
    results.append(p)
    print("Finished model: " + str(n))


In [ ]:
# Plot results as a function of  epochs to see if a certain nr of nodes makes a difference. 

plt.figure(figsize = (20,20))

counter = 0
for model in results:
    mse_train = model.eval_['train_preform']
    mse_test = model.eval_['valid_preform']
    cost_train = model.eval_["cost_train"]
    r2_test = model.eval_['valid_preform_r2']

    ax1 = plt.subplot(211)
    plt.plot(r2_test, label = str(n_nodes[counter])) 
    plt.title("R2 score for different network architectures", fontsize = 20)
    plt.ylabel("R2 score", fontsize = 15)
    plt.legend()
    
    # Alle disse blir like.
    ax2 = plt.subplot(212)
    plt.plot(mse_test, label = str(n_nodes[counter])) 
    plt.title("MSE test for different one hidden layer architectures", fontsize = 20)
    plt.ylabel("MSE", fontsize = 15)
    plt.legend()
    
    counter += 1

plt.xlabel("Epochs", fontsize = 15)
plt.savefig("results/figures/NN_one_hidden_layer_elu_1000epochs_eta0.0001_featured_scaled_7.png")

In [ ]:
error = results[7].model_error
plt.hist(error)
plt.title("Histogram error using elu, 1000 epochs and one timestep of data", fontsize = 15)
plt.savefig("results/figures/NN_one_hidden_layer_elu_1000epochs_eta0.0001_featured_scaled_1_error.png")
# They all look the same. 

# Running two hidden layers - combinations in [100, 125, 150, 175, 200]
## Ran fro all permutations (includes ordering) didn't give a large manifestations ~0.01

In [ ]:
n_1 = [100, 125]
n_2 = [100, 150]
n_7 = [100, 175]
n_3 = [100, 200]

n_4 = [125, 150]
n_8 = [125, 175]
n_5 = [125, 200]

n_9 = [150, 175]
n_6 = [150, 200]

n_10 = [175,200]

results = []

#permutations_of_nodes = list(itertools.permutations(n_1)) + list(itertools.permutations(n_2)) \
#+ list(itertools.permutations(n_3))+ list(itertools.permutations(n_4)) + list(itertools.permutations(n_5))+ \
#list(itertools.permutations(n_6)) + list(itertools.permutations(n_7)) + list(itertools.permutations(n_8)) +list(itertools.permutations(n_9)) + list(itertools.permutations(n_10))

combinations_of_nodes = comb = [n_1, n_2, n_3, n_4, n_5, n_6, n_7, n_8, n_9, n_10]

for per in combinations_of_nodes:
    model = NeuralNetRegressor(n_hidden = per,  
                               epochs=1000, 
                               eta=0.0001,
                               shuffle=True, 
                               batch_size=10,
                               seed=None, 
                               alpha=0.0001, # Dette er for relu ikke det samme som penaltien i scikit learn  
                               activation='sigmoid')

    p = model.fit(X_train, y_train, X_test, y_test)
    model.predict(X_test)
    results.append(p)
    print("finished model: " + str(per) + " performance " + str(np.nanmean(model.eval_["valid_preform"])) )


In [ ]:
plt.figure(figsize = (20,20))

counter = 0
for model in results:
    mse_test = model.eval_['valid_preform']
    r2_train = model.eval_['valid_preform_r2']

    ax1 = plt.subplot(211)
    plt.plot(r2_train[:200], label = str(combinations_of_nodes[counter])) 
    plt.title("R2 score for different neural network architectures", fontsize = 20)
    plt.ylabel("R2 score", fontsize = 15)
    plt.legend()
    
    # Alle disse blir like.
    ax2 = plt.subplot(212)
    plt.plot(mse_test[:200], label = str(combinations_of_nodes[counter])) 
    plt.title("MSE test for different neural network architectures", fontsize = 20)
    plt.ylabel("MSE", fontsize = 15)
    plt.legend()
    
    counter += 1

plt.xlabel("Epochs", fontsize = 15)
plt.savefig("results/figures/NN_two_hidden_layer_sigmoid_1000epochs_eta0.0001_StandardScaler_mse_r2_1_200epochs_7.png")

In [ ]:
error = results[0].model_error
plt.hist(error)
plt.title("Histogram error using sigmoid, two layers, 1000 epochs and one timestep of data", fontsize = 15)
plt.savefig("results/figures/NN_two_hidden_layer_sigmoid_1000epochs_eta0.0001_featured_scaled_1_error.png")
# They all look the same. 

# Neural network three hidden layers

In [ ]:
n_1 = [100, 125, 150]
n_2 = [100, 125, 175]
n_3 = [100, 125, 200]

n_4 = [100, 150, 175]
n_5 = [100, 150, 200]

n_6 = [100, 175, 200]

n_7 = [125, 150, 175]
n_8 = [125, 175, 200]

n_9 = [150, 175, 200]

comb = [n_1, n_2, n_3, n_4, n_5, n_6, n_7, n_8, n_9]


results = []

#permutations_of_nodes = list(itertools.permutations(n_1)) + list(itertools.permutations(n_2)) \
#+ list(itertools.permutations(n_3))+ list(itertools.permutations(n_4)) + list(itertools.permutations(n_5))+ \
#list(itertools.permutations(n_6)) + list(itertools.permutations(n_7)) + list(itertools.permutations(n_8)) +list(itertools.permutations(n_9)) 
    
for per in comb:
    model = NeuralNetRegressor(n_hidden = per,  
                               epochs=100, 
                               eta=0.0001,
                               shuffle=True, 
                               batch_size=10,
                               seed=None, 
                               alpha=0.0001, # Dette er for relu ikke det samme som penaltien i scikit learn  
                               activation='sigmoid')

    p = model.fit(X_train, y_train, X_test, y_test)
    model.predict(X_test)
    results.append(p)
    print("finished model: " + str(per) + " performance " + str(np.nanmean(model.eval_["valid_preform"])) )

In [ ]:
plt.figure(figsize = (20,20))

counter = 0
for model in results:
    mse_test = model.eval_['valid_preform']
    r2_train = model.eval_['valid_preform_r2']

    ax1 = plt.subplot(211)
    plt.plot(r2_train, label = str(comb[counter])) 
    plt.title("R2 score for different neural network architectures", fontsize = 20)
    plt.ylabel("R2 score", fontsize = 15)
    plt.legend()
    
    # Alle disse blir like.
    ax2 = plt.subplot(212)
    plt.plot(mse_test, label = str(comb[counter])) 
    plt.title("MSE test for different neural network architectures", fontsize = 20)
    plt.ylabel("MSE", fontsize = 15)
    plt.legend()
    
    counter += 1

plt.xlabel("Epochs", fontsize = 15)
plt.savefig("results/figures/NN_three_hidden_layer_sigmoid_1000epochs_eta0.0001_StandardScaler_mse_r2_7.png")

In [ ]:
error = results[0].model_error
plt.hist(error)
plt.title("Histogram error using sigmoid, three layers, 1000 epochs and one timestep of data", fontsize = 15)
plt.savefig("results/figures/NN_three_hidden_layer_sigmoid_1000epochs_eta0.0001_featured_scaled_1_error.png")
# They all look the same.

# Net med fire hidden layers

In [ ]:
n_1 = [100, 125, 150, 175]
n_2 = [100, 125, 150, 200]
n_3 = [125, 150, 175, 200]

results = []

#permutations_of_nodes = list(itertools.permutations(n_1)) + list(itertools.permutations(n_2)) + list(itertools.permutations(n_3))
comb = [n_1, n_2, n_3]

for per in comb:
    model = NeuralNetRegressor(n_hidden = per,  
                               epochs=100, 
                               eta=0.0001,
                               shuffle=True, 
                               batch_size=10,
                               seed=None, 
                               alpha=0.0001, # Dette er for relu ikke det samme som penaltien i scikit learn  
                               activation='sigmoid')

    p = model.fit(X_train, y_train, X_test, y_test)
    model.predict(X_test)
    results.append(p)
    print("finished model: " + str(per) + " performance " + str(np.nanmean(model.eval_["valid_preform"])) )

In [ ]:
plt.figure(figsize = (20,20))

counter = 0
for model in results:
    mse_test = model.eval_['valid_preform']
    r2_train = model.eval_['valid_preform_r2']

    ax1 = plt.subplot(211)
    plt.plot(r2_train, label = str(comb[counter])) 
    plt.title("R2 score for different neural network architectures", fontsize = 20)
    plt.ylabel("R2 score", fontsize = 15)
    plt.legend()
    
    # Alle disse blir like.
    ax2 = plt.subplot(212)
    plt.plot(mse_test, label = str(comb[counter])) 
    plt.title("MSE test for different neural network architectures", fontsize = 20)
    plt.ylabel("MSE", fontsize = 15)
    plt.legend()
    
    counter += 1

plt.xlabel("Epochs", fontsize = 15)
plt.savefig("results/figures/NN_four_hidden_layer_sigmoid_1000epochs_eta0.0001_StandardScaler_mse_r2_7.png")

In [ ]:
error = results[0].model_error
plt.hist(error)
plt.title("Histogram error using sigmoid, four layers, 1000 epochs and one timestep of data", fontsize = 15)
plt.savefig("results/figures/NN_four_hidden_layer_sigmoid_1000epochs_eta0.0001_featured_scaled_1_error.png")
# They all look the same.

# Permutations of the best four layer model

In [ ]:
n_1 = [100, 125, 150, 175]
n_2 = [100, 125, 150, 200]
n_3 = [125, 150, 175, 200]

results = []

permutations_of_nodes = list(itertools.permutations(n_3)) #+ list(itertools.permutations(n_2)) + list(itertools.permutations(n_3))
#comb = [n_1, n_2, n_3]

for per in permutations_of_nodes:
    model = NeuralNetRegressor(n_hidden = list(per),  
                               epochs=100, 
                               eta=0.0001,
                               shuffle=True, 
                               batch_size=10,
                               seed=None, 
                               alpha=0.0001, # Dette er for relu ikke det samme som penaltien i scikit learn  
                               activation='sigmoid')

    p = model.fit(X_train, y_train, X_test, y_test)
    model.predict(X_test)
    results.append(p)
    print("finished model: " + str(per) + " performance " + str(np.nanmean(model.eval_["valid_preform"])) )

In [ ]:
plt.figure(figsize = (20,20))

counter = 0
for model in results:
    mse_test = model.eval_['valid_preform']
    r2_train = model.eval_['valid_preform_r2']

    ax1 = plt.subplot(211)
    plt.plot(r2_train, label = str(permutations_of_nodes[counter])) 
    plt.title("R2 score for different neural network architectures", fontsize = 20)
    plt.ylabel("R2 score", fontsize = 15)
    plt.legend()
    
    # Alle disse blir like.
    ax2 = plt.subplot(212)
    plt.plot(mse_test, label = str(permutations_of_nodes[counter])) 
    plt.title("MSE test for different neural network architectures", fontsize = 20)
    plt.ylabel("MSE", fontsize = 15)
    plt.legend()
    
    counter += 1

plt.xlabel("Epochs", fontsize = 15)
plt.savefig("results/figures/NN_four_hidden_layer_sigmoid_1000epochs_eta0.0001_StandardScaler_mse_r2_permutationd_7.png")

# Scikit MLP Regressor using one hidden layer

In [ ]:
#n_nodes = [10,30,50,100, 500]
"""
eta = [0.0001, 0.001, 0.01, 0.1, 1.0]
lmd = [0.0001, 0.001, 0.01, 0.1, 1.0, 10]
#epochs = [10,50,100] Scikit stopper på rett antall epochs
batch_s = [1,10,50]

# Scikit uses a different shape than we have implemented in our neural network.
y_train = y_train.ravel()




#for e in epochs:

for b in batch_s:
    for et in eta:
        for l in lmd:
            mlp = MLPRegressor(hidden_layer_sizes=(30,), 
                               activation = 'relu', # identity’, ‘logistic’, ‘tanh’, ‘relu’
                               solver = "adam", 
                               alpha = l, # penalty
                               batch_size = b, 
                               learning_rate_init = et)

            mlp.fit(X_train, y_train)
            y_pred = mlp.predict(X_test)
            #logistic activation uses the sigmoid function 
            
            
            # NEED TO TRANSFORM AND NORMALIZE THE RESULT. 
            #mse = NRMSE(y_pred, y_test)
            #ajusted_r2 = A_R2(y_pred, y_test, n, p)
            print(" eta : " + str(et) + " lmd "+ str(l) +"    batch size : " +  str(b)   + "   mse is " + str(mse) + " a r2  " + str(ajusted_r2))
            """